# Notebook - v3

Notebook that incorporates Kevin work on preprocessing data and for testing models

In [13]:
%load_ext autoreload
%autoreload 2


### Model baseline testing

In [4]:
from sentinelhub import SentinelHubRequest, DataCollection, MimeType, CRS, BBox, SHConfig
from dotenv import load_dotenv
from pyproj import Transformer
import os
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import folium
import cv2
from urban_watch.ml_logic.data import load_data
from s2cloudless import S2PixelCloudDetector
from urban_watch.ml_logic.package import CloudMasker
from urban_watch.ml_logic.package import preprocess_image
import numpy as np
import pandas as pd

from urban_watch.ml_logic.data import load_data
from urban_watch.ml_logic.labels import (
    get_bbox_from_features,
    bbox_to_wgs84,
    tile_name_from_bbox_wgs84,
    get_label_array
)
from urban_watch.ml_logic.package import CloudMasker, DataCleaner, IndexCalculator

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report

/Users/clairedebadts/.pyenv/versions/urban_watch/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.6) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)


In [5]:
from sentinelhub import SHConfig
from dotenv import load_dotenv
import os

In [6]:
load_dotenv()  # charge automatiquement le .env du dossier courant

config = SHConfig()
config.sh_client_id = os.environ.get("SH_CLIENT_ID")
config.sh_client_secret = os.environ.get("SH_CLIENT_SECRET")

In [7]:
import os
print(os.getenv("BUCKET_NAME"))

urbanwatch-labels


Cellule 1 — Imports + chargement des X & Y

In [8]:
# 1 Charger les features X (tuiles Sentinel)
X_array, meta = load_data()
print("✔ X loaded:", X_array.shape)  # (n_tiles, 300, 300, 10)

#  Charger les bbox pour aller chercher les Y
bbox_x, crs_x = get_bbox_from_features()
bbox_wgs = bbox_to_wgs84(bbox_x, crs_x)
tile_names = tile_name_from_bbox_wgs84(bbox_wgs)

# 3) Charger les labels WorldCover (tuiles Y 300x300)
Y_tiles = get_label_array(tile_names, bbox_wgs, bbox_x, crs_x)
Y_array = np.stack(Y_tiles, axis=0)  # (n_tiles, 300, 300)

print("✔ Y loaded:", Y_array.shape)

✔ X loaded: (10, 300, 300, 10)
✔ Y loaded: (10, 300, 300)


Cellule 2 — Conversion des labels en binaire (urban / non urban)

In [9]:
def worldcover_to_binary(tile, urban_classes={50}):
    """
    tile : array (H, W) avec codes ESA WorldCover
    urban_classes : set des codes considérés comme urban

    Retourne : array (H, W) binaire {0,1}
    """
    mask_urban = np.isin(tile, list(urban_classes))
    return mask_urban.astype(int)

Y_binary = np.stack([worldcover_to_binary(t) for t in Y_array], axis=0)

print("Y_binary shape:", Y_binary.shape)
print("Unique values:", np.unique(Y_binary, return_counts=True))


Y_binary shape: (10, 300, 300)
Unique values: (array([0, 1]), array([529317, 370683]))


Cellule 3 — Préprocessing par tuile

In [10]:
def preprocess_image_for_baseline(img):
    """
    img : (H, W, 10) brut Sentinel-2

    Retourne :
      - img_std : (H, W, 13)  -> 10 bandes normalisées + 3 indices (ndvi, ndbi, mndwi)
      - mask_valid : (H, W) bool -> pixels utilisables (pas NaN)
    """
    cleaner = DataCleaner()
    cloud_detector = CloudMasker()

    # 1) Normalisation des bandes (0-1)
    img_norm = cleaner.normalize_bands(img)

    # 2) Détection nuages
    cloud_mask = cloud_detector.detect_clouds(img_norm)

    # 3) Masquage nuages (nan)
    img_masked = cloud_detector.apply_mask(img_norm, cloud_mask, fill_value=np.nan)

    # 4) Indices spectraux
    B3  = img_masked[:, :, CloudMasker.BAND_IDX["B03"]]
    B4  = img_masked[:, :, CloudMasker.BAND_IDX["B04"]]
    B8  = img_masked[:, :, CloudMasker.BAND_IDX["B08"]]
    B11 = img_masked[:, :, CloudMasker.BAND_IDX["B11"]]

    ndvi  = IndexCalculator.ndvi(B4,  B8)
    ndbi  = IndexCalculator.ndbi(B11, B8)
    mndwi = IndexCalculator.mndwi(B3,  B11)

    ndvi  = ndvi[..., np.newaxis]
    ndbi  = ndbi[..., np.newaxis]
    mndwi = mndwi[..., np.newaxis]

    # 5) Concaténer 10 bandes + 3 indices = 13 canaux
    img_13 = np.concatenate([img_masked, ndvi, ndbi, mndwi], axis=-1)

    # 6) Standardisation (0 mean / 1 std par bande)
    img_std = cleaner.standardize(img_13)

    # 7) Mask valid (aucun NaN sur les 13 canaux)
    mask_valid = ~np.isnan(img_std).any(axis=-1)

    return img_std, mask_valid


In [11]:
# Appliquer à toutes les tuiles
X_processed = []
valid_masks = []

for img in X_array:
    img_std, mask_valid = preprocess_image_for_baseline(img)
    X_processed.append(img_std)
    valid_masks.append(mask_valid)

X_processed = np.stack(X_processed, axis=0)   # (n_tiles, H, W, 13)
valid_masks = np.stack(valid_masks, axis=0)   # (n_tiles, H, W)

print("X_processed:", X_processed.shape)
print("valid_masks:", valid_masks.shape)


X_processed: (10, 300, 300, 13)
valid_masks: (10, 300, 300)


Cellule 4 — Construction du dataset pixel-wise (X_pixels, y_pixels)

In [12]:
X_pixels_list = []
y_pixels_list = []

n_tiles, H, W, C = X_processed.shape

for i in range(n_tiles):
    img13 = X_processed[i]      # (H, W, 13)
    mask  = valid_masks[i]      # (H, W)
    y_tile = Y_binary[i]        # (H, W)

    # On ne garde que les pixels valides
    valid = mask

    X_pix = img13[valid]        # (n_valid_pixels, 13)
    y_pix = y_tile[valid]       # (n_valid_pixels,)

    X_pixels_list.append(X_pix)
    y_pixels_list.append(y_pix)

X_pixels = np.concatenate(X_pixels_list, axis=0)
y_pixels = np.concatenate(y_pixels_list, axis=0)

print("X_pixels:", X_pixels.shape)
print("y_pixels:", y_pixels.shape)
print("Classe distribution:", np.unique(y_pixels, return_counts=True))


X_pixels: (449451, 13)
y_pixels: (449451,)
Classe distribution: (array([0, 1]), array([300964, 148487]))


In [13]:
print("NaN in X_pixels:", np.isnan(X_pixels).sum())

NaN in X_pixels: 0


In [22]:
print("NaN in y_pixels:", np.isnan(y_pixels).sum())

NaN in y_pixels: 0


Cellule 5 — Train / Test split pixel-wise

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pixels, y_pixels,
    test_size=0.2,
    random_state=42,
    stratify=y_pixels 
)

print("Train size:", X_train.shape[0])
print("Test size:", X_test.shape[0])
print("Train distribution:", np.unique(y_train, return_counts=True))
print("Test distribution:", np.unique(y_test, return_counts=True))


Train size: 359560
Test size: 89891
Train distribution: (array([0, 1]), array([240771, 118789]))
Test distribution: (array([0, 1]), array([60193, 29698]))


Cellule 6 — Modèles baseline

In [20]:
# Logistic regression

log_reg = LogisticRegression(max_iter=500, class_weight="balanced")

log_reg.fit(X_train, y_train)
y_pred_lr = log_reg.predict(X_test)

print("LogReg accuracy:", accuracy_score(y_test, y_pred_lr))
print("LogReg f1-score:", f1_score(y_test, y_pred_lr))
print("\nClassification report (LogReg):\n")
print(classification_report(y_test, y_pred_lr))

LogReg accuracy: 0.6920270104904829
LogReg f1-score: 0.6021384840907131

Classification report (LogReg):

              precision    recall  f1-score   support

           0       0.83      0.69      0.75     60193
           1       0.53      0.71      0.60     29698

    accuracy                           0.69     89891
   macro avg       0.68      0.70      0.68     89891
weighted avg       0.73      0.69      0.70     89891



In [18]:
# Random Forest

rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    n_jobs=-1,
    random_state=42
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print("RF accuracy:", accuracy_score(y_test, y_pred_rf))
print("RF f1-score:", f1_score(y_test, y_pred_rf))
print("\nClassification report (RandomForest):\n")
print(classification_report(y_test, y_pred_rf))


RF accuracy: 0.8656706455596223
RF f1-score: 0.7850237675586177

Classification report (RandomForest):

              precision    recall  f1-score   support

           0       0.88      0.93      0.90     60193
           1       0.83      0.74      0.79     29698

    accuracy                           0.87     89891
   macro avg       0.86      0.83      0.84     89891
weighted avg       0.86      0.87      0.86     89891



In [21]:
# History Gradient Boosting Classifier

hgb = HistGradientBoostingClassifier(
    max_depth=8,
    learning_rate=0.1,
    l2_regularization=0.1,
    max_bins=255
)

hgb.fit(X_train, y_train)
y_pred_hgb = hgb.predict(X_test)

print("HGB accuracy:", accuracy_score(y_test, y_pred_hgb))
print("HGB f1-score:", f1_score(y_test, y_pred_hgb))
print("\nClassification report (HGB):\n")
print(classification_report(y_test, y_pred_hgb))


HGB accuracy: 0.8138968306059561
HGB f1-score: 0.6944307450636564

Classification report (HGB):

              precision    recall  f1-score   support

           0       0.84      0.90      0.87     60193
           1       0.76      0.64      0.69     29698

    accuracy                           0.81     89891
   macro avg       0.80      0.77      0.78     89891
weighted avg       0.81      0.81      0.81     89891



In [24]:
"""
model.py — Baseline Random Forest model for UrbanWatch
-------------------------------------------------------

Compatible with:
- data.py (loading X tiles)
- labels.py (loading Y tiles)
- package.py (preprocess_image => 13 standardized bands)

Pipeline:
    X_array        = (n_tiles, 300,300,10)
    X_processed    = list of (n_valid_pixels, 13)
    valid_masks    = list of (300,300)
    Y_tiles        = list of (300,300)

Output: pixel-level RandomForest model.
"""

import os
import joblib
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report


# ============================================================
# 1. Build pixel-level dataset
# ============================================================

def build_pixel_dataset(x_processed_list, y_tiles, valid_masks):
    """
    Convert tiles into supervised pixel-level dataset.

    Args:
        x_processed_list (list): list of arrays (n_valid_pixels, 13)
        y_tiles (list): list of (300,300) label tiles
        valid_masks (list): list of boolean masks (300,300)

    Returns:
        tuple:
            x_pixels (np.ndarray): (N, 13)
            y_pixels (np.ndarray): (N,)
    """
    x_pixels = []
    y_pixels = []

    for x_tile, y_tile, mask in zip(x_processed_list, y_tiles, valid_masks):
        y_valid = y_tile[mask]
        x_pixels.append(x_tile)
        y_pixels.append(y_valid)

    x_pixels = np.vstack(x_pixels)
    y_pixels = np.hstack(y_pixels)

    return x_pixels, y_pixels


# ============================================================
# 2. Stratified split
# ============================================================

def split_data(x_data, y_data, test_size=0.2, seed=42):
    """
    Stratified train/test split.

    Args:
        x_data (np.ndarray): feature matrix
        y_data (np.ndarray): labels
        test_size (float): test proportion
        seed (int): random seed

    Returns:
        tuple: x_train, x_test, y_train, y_test
    """
    return train_test_split(
        x_data,
        y_data,
        test_size=test_size,
        random_state=seed,
        stratify=y_data,
    )


# ============================================================
# 3. Train Random Forest
# ============================================================

def train_random_forest(x_train, y_train):
    """
    Train baseline RandomForest model.

    Args:
        x_train (np.ndarray): training features
        y_train (np.ndarray): training labels

    Returns:
        RandomForestClassifier: trained model
    """
    model = RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_split=4,
        min_samples_leaf=2,
        bootstrap=True,
        n_jobs=-1,
    )
    model.fit(x_train, y_train)
    return model


# ============================================================
# 4. Evaluation metrics
# ============================================================

def evaluate(model, x_test, y_test):
    """
    Compute accuracy, F1-score and detailed classification report.

    Args:
        model: trained model
        x_test (np.ndarray): test features
        y_test (np.ndarray): test labels

    Returns:
        tuple: accuracy, f1_score, classification_report (str)
    """
    y_pred = model.predict(x_test)

    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print("\n--- Evaluation ---")
    print("Accuracy:", acc)
    print("F1-score:", f1)
    print(report)

    return acc, f1, report


# ============================================================
# 5. Save / Load model
# ============================================================

def save_model(model, path="model_random_forest.joblib"):
    """
    Save trained model to disk.

    Args:
        model: trained model
        path (str): file path
    """
    directory = os.path.dirname(path)
    if directory:
        os.makedirs(directory, exist_ok=True)

    joblib.dump(model, path)
    print(f"✔ Model saved to: {path}")


def load_model(path="model_random_forest.joblib"):
    """
    Load model from disk.

    Args:
        path (str): model file path

    Returns:
        model: loaded model
    """
    model = joblib.load(path)
    print(f"✔ Model loaded from: {path}")
    return model


# ============================================================
# 6. Predict on a full tile
# ============================================================

def predict_tile(model, x_processed, valid_mask):
    """
    Predict classes for every valid pixel in a tile.

    Args:
        model: trained classifier
        x_processed (np.ndarray): (n_valid_pixels, 13)
        valid_mask (np.ndarray): (300,300) boolean mask

    Returns:
        np.ndarray: (300,300) map with values {-1,0,1}
    """
    preds = model.predict(x_processed)

    output = np.full(valid_mask.shape, -1, dtype=np.int8)
    output[valid_mask] = preds

    return output
